In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [9]:
tf.keras.backend.clear_session()

In [10]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [11]:
subject = 'Pernambuco - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [12]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Produção de Cimento (t),Pernambuco - Consumo de Cimento (t)
0,2008,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,0.704795,0.787611,0.660899,5.483335e+08,2.546644e+09,...,0.671922,23.704331,1012.356520,1564.281549,25.868467,21.249870,75.428608,2.118063,722.146507,1717.153000
1,2009,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,0.707057,0.790400,0.665481,5.302764e+08,2.679945e+09,...,0.671804,22.861798,1012.209502,1487.098457,25.873200,21.577129,77.771392,1.951534,803.348817,1878.938000
2,2010,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,0.709020,0.792674,0.669877,5.046425e+08,2.891930e+09,...,0.671411,22.487970,1012.234225,1437.305833,25.750070,21.891970,80.133165,1.854189,950.406487,2406.931000
3,2011,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,0.710693,0.794444,0.674084,4.854622e+08,3.091988e+09,...,0.670748,21.107295,1012.512701,1436.247207,25.736127,21.841067,80.179132,1.815199,1079.328424,2496.113000
4,2012,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,0.711600,0.795572,0.677443,4.867661e+08,3.189506e+09,...,0.669292,21.566683,1012.689602,1436.457629,25.711228,21.251350,77.643662,1.728413,1132.134532,2728.230000
5,2013,9.627386e+07,5.616792e+06,8.939248,7.935398e+07,0.713567,0.796536,0.682563,5.225848e+08,3.093872e+09,...,0.668984,21.669594,1012.907542,1521.117838,25.685608,20.953182,76.283825,1.611013,1107.644159,2982.021000
6,2014,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.714447,0.796914,0.686026,5.676413e+08,2.779340e+09,...,0.667649,19.852601,1013.159804,1654.440407,25.780576,20.818540,75.113876,1.558118,988.207737,2983.632857
7,2015,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.714401,0.796740,0.688529,5.179392e+08,2.561731e+09,...,0.665120,18.352626,1013.187593,1731.551083,25.909409,21.066299,75.720358,1.597579,808.858465,2632.837886
8,2016,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.722910,0.809069,0.701506,4.524144e+08,2.516859e+09,...,0.666277,16.821327,1013.081354,1774.985187,25.897163,21.315125,76.984087,1.606812,597.562419,2308.455257
9,2017,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.722811,0.808712,0.702551,4.213689e+08,2.497710e+09,...,0.665302,16.755645,1013.025277,1768.623220,25.854367,21.296659,77.129112,1.578862,438.177864,1813.941000


In [13]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Produção de Cimento (t)
0,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,0.704795,0.787611,0.660899,5.483335e+08,2.546644e+09,1.874883e+10,...,8.049769,0.671922,23.704331,1012.356520,1564.281549,25.868467,21.249870,75.428608,2.118063,722.146507
1,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,0.707057,0.790400,0.665481,5.302764e+08,2.679945e+09,2.055015e+10,...,8.078293,0.671804,22.861798,1012.209502,1487.098457,25.873200,21.577129,77.771392,1.951534,803.348817
2,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,0.709020,0.792674,0.669877,5.046425e+08,2.891930e+09,2.342247e+10,...,8.042187,0.671411,22.487970,1012.234225,1437.305833,25.750070,21.891970,80.133165,1.854189,950.406487
3,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,0.710693,0.794444,0.674084,4.854622e+08,3.091988e+09,2.604265e+10,...,8.028882,0.670748,21.107295,1012.512701,1436.247207,25.736127,21.841067,80.179132,1.815199,1079.328424
4,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,0.711600,0.795572,0.677443,4.867661e+08,3.189506e+09,2.708753e+10,...,12.192814,0.669292,21.566683,1012.689602,1436.457629,25.711228,21.251350,77.643662,1.728413,1132.134532
5,9.627386e+07,5.616792e+06,8.939248,7.935398e+07,0.713567,0.796536,0.682563,5.225848e+08,3.093872e+09,2.523394e+10,...,13.117303,0.668984,21.669594,1012.907542,1521.117838,25.685608,20.953182,76.283825,1.611013,1107.644159
6,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.714447,0.796914,0.686026,5.676413e+08,2.779340e+09,1.954361e+10,...,13.857509,0.667649,19.852601,1013.159804,1654.440407,25.780576,20.818540,75.113876,1.558118,988.207737
7,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.714401,0.796740,0.688529,5.179392e+08,2.561731e+09,1.327596e+10,...,14.543115,0.665120,18.352626,1013.187593,1731.551083,25.909409,21.066299,75.720358,1.597579,808.858465
8,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.722910,0.809069,0.701506,4.524144e+08,2.516859e+09,1.011370e+10,...,14.833751,0.666277,16.821327,1013.081354,1774.985187,25.897163,21.315125,76.984087,1.606812,597.562419
9,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.722811,0.808712,0.702551,4.213689e+08,2.497710e+09,8.456309e+09,...,14.580259,0.665302,16.755645,1013.025277,1768.623220,25.854367,21.296659,77.129112,1.578862,438.177864


In [14]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1878.938000
1     2406.931000
2     2496.113000
3     2728.230000
4     2982.021000
5     2983.632857
6     2632.837886
7     2308.455257
8     1813.941000
9     1645.004000
10    1690.846000
11    2018.834000
12    2022.411000
13    1940.927000
14            NaN
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [15]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Produção de Cimento (t)
0,-1.894045,1.016308,-1.508819,-2.131804,-1.583249,-1.530584,-1.598164,1.204752,-0.990234,0.166091,...,-1.373812,1.355095,1.532738,-1.179403,-0.231175,0.290203,-0.299372,-1.157338,1.628734,-0.228478
1,-1.694920,1.229887,-0.890758,-1.702987,-1.220442,-1.140159,-1.290136,0.820657,-0.546688,0.428911,...,-1.363879,1.312892,1.177660,-1.613673,-0.847777,0.336507,0.759984,0.359687,0.803272,0.075034
2,-1.063798,1.314665,0.449724,-0.423797,-0.905536,-0.821949,-0.994665,0.275392,0.158672,0.847995,...,-1.376452,1.172184,1.020114,-1.540643,-1.245561,-0.868108,1.779145,1.889008,0.320744,0.624695
3,-0.515495,1.139248,1.171687,0.366041,-0.637043,-0.574308,-0.711867,-0.132596,0.824346,1.230291,...,-1.381086,0.934380,0.438242,-0.718070,-1.254018,-1.004513,1.614369,1.918773,0.127477,1.106570
4,-0.028247,0.749420,1.471589,0.937804,-0.491590,-0.416427,-0.486099,-0.104860,1.148829,1.382743,...,0.068995,0.412716,0.631847,-0.195530,-1.252337,-1.248108,-0.294581,0.276978,-0.302714,1.303944
5,0.353986,0.225440,1.211705,1.102177,-0.176084,-0.281472,-0.141927,0.657046,0.830615,1.112297,...,0.390946,0.301975,0.675218,0.448231,-0.576002,-1.498756,-1.259768,-0.603559,-0.884651,1.212406
6,0.637781,-0.389615,0.795877,0.989019,-0.034819,-0.228569,0.090888,1.615453,-0.215960,0.282052,...,0.648721,-0.176426,-0.090537,1.193372,0.489088,-0.569662,-1.695614,-1.361137,-1.146844,0.765986
7,0.807393,-0.839081,0.097662,0.625383,-0.042209,-0.252893,0.259082,0.558230,-0.940033,-0.632425,...,0.887482,-1.082994,-0.722687,1.275456,1.105111,0.690753,-0.893603,-0.968421,-0.951240,0.095628
8,0.945883,-1.118984,-0.131209,0.524538,1.322915,1.472501,1.131361,-0.835563,-1.089341,-1.093812,...,0.988695,-0.668215,-1.368038,0.961643,1.452098,0.570944,-0.088137,-0.150117,-0.905473,-0.694139
9,0.973351,-1.179977,-0.300688,0.396558,1.307025,1.422494,1.201669,-1.495939,-1.153056,-1.335633,...,0.900417,-1.017629,-1.395719,0.796001,1.401274,0.152257,-0.147914,-0.056209,-1.044017,-1.289874


In [16]:
reshaped_train_input = dfToInputRNN(train_input)

In [17]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.63530724, -0.95624748, -1.14206076, -0.46989042,
          1.08697714,  1.02203982,  1.12974224, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          0.74875685, -1

In [18]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     1878.938000
1     2406.931000
2     2496.113000
3     2728.230000
4     2982.021000
5     2983.632857
6     2632.837886
7     2308.455257
8     1813.941000
9     1645.004000
10    1690.846000
11    2018.834000
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [19]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    2018.834
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [20]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Produção de Cimento (t)
12,0.243152,-0.752383,-1.854042,-1.166663,0.818285,0.699365,1.038092,1.108405,2.635823,-0.189959,...,0.742044,-1.580713,-1.117166,-0.348766,-0.031355,0.055743,1.110593,1.231105,0.854739,-1.391209


In [21]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.24315161, -0.7523835 , -1.85404211, -1.16666318,
          0.81828459,  0.69936487,  1.03809159,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          0.74204352, -1

In [22]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2022.411
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [23]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.002
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.002)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [24]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [25]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[419456516, 972915026, 3777908091, 4019380187, 4172451477, 664087539, 4260212390, 573666585, 1137346003, 1143083734, 3046961286, 3482672458, 2538503142, 4184299975, 2087849639, 2957650855, 1082612832, 1122995926, 1303983395, 326540637, 3293370904, 4002731977, 1287497697, 3681588776, 2849680584, 259259900, 221294136, 2081689381, 1831565305, 220061717, 235476845, 4237986883, 1937365059, 3247592368, 2690795992, 1934327810, 1443354042, 3632469474, 1708692134, 2388015074, 537991153, 1758063853, 1010169431, 1555379354, 3345383822, 1467850401, 2260920006, 2857151551, 2877686524, 3887589303, 4206968084, 3467544188, 2419863335, 2527753222, 664886627, 1153460614, 3446897387, 2594407784, 501642749, 1137461724, 4249029325, 2089589492, 2039575304, 2803159580, 3703200552, 3714750596, 1650825445, 2265676497, 2160415030, 2640815009, 4039079033, 247325992, 4182743910, 3422143314, 1001241343, 2524644224, 3952057753, 3806216747, 2062183462, 730650407, 2705216140, 3402114414, 1564138897, 2353085534, 23654

2023-09-09 13:00:20.305782: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 13:00:20.325729: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 13:00:20.325918: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 4912.67529296875
winner_seed: 419456516


Step: 1 ___________________________________________
loss: 2840.6953125
winner_seed: 972915026


Step: 2 ___________________________________________
loss: 2384.477783203125
winner_seed: 3777908091


Step: 3 ___________________________________________
loss: 3939.657958984375


Step: 4 ___________________________________________
loss: 2782.749755859375


Step: 5 ___________________________________________
loss: 4224.69482421875


Step: 6 ___________________________________________
loss: 3781.7197265625


Step: 7 ___________________________________________
loss: 2607.854248046875


Step: 8 ___________________________________________
loss: 1483.7813720703125
winner_seed: 1137346003


Step: 9 ___________________________________________
loss: 1908.2698974609375


Step: 10 ___________________________________________
loss: 1763.7464599609375


Step: 11 ___________________________________________
loss: 2528.118896484375


Step: 12 _________________

In [26]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 5498733.5000
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 5450279.5000
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 5272362.5000
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 3838579.2500
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 1120451.2500
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 323030.7500
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 239935.3906
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 220092.0469
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 221500.9531
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 177037.4219
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 108254.8672
Epoch 12/10000
1/1 [==============================] - 0s 4m

1/1 [==============================] - 0s 4ms/step - loss: 26959.2012
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 20386.1074
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 20439.8066
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 20856.2617
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 15655.7812
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 28959.3984
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 37549.3594
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 17352.9160
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 40829.9883
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 19978.6816
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 25367.7129
Epoch 108/10000
1/1 [==============================] - 0s 5ms/step - los

1/1 [==============================] - 0s 3ms/step - loss: 18245.5176
Epoch 193/10000
1/1 [==============================] - 0s 4ms/step - loss: 33047.5469
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 24133.1934
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 22240.1582
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 14248.9150
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 9976.0234
Epoch 198/10000
1/1 [==============================] - 0s 3ms/step - loss: 25039.2910
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 26604.4043
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 17056.2656
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 16728.1484
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 16519.8848
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - lo

1/1 [==============================] - 0s 6ms/step - loss: 35712.2617
Epoch 288/10000
1/1 [==============================] - 0s 6ms/step - loss: 37362.0703
Epoch 289/10000
1/1 [==============================] - 0s 6ms/step - loss: 20033.5996
Epoch 290/10000
1/1 [==============================] - 0s 6ms/step - loss: 22839.7500
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 24411.2422
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 23480.0254
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 9860.8721
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 22733.6191
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 25116.8926
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 9307.9658
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 13402.4756
Epoch 298/10000
1/1 [==============================] - 0s 5ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 11426.4697
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 14252.9961
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 21105.7676
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 11988.5752
Epoch 386/10000
1/1 [==============================] - 0s 3ms/step - loss: 11367.6143
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 12749.1631
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 11086.2305
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 17282.4258
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 21087.3457
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 10093.3311
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 26770.9785
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - l

...

1/1 [==============================] - 0s 4ms/step - loss: 12795.3818
Epoch 573/10000
1/1 [==============================] - 0s 4ms/step - loss: 11434.2920
Epoch 574/10000
1/1 [==============================] - 0s 4ms/step - loss: 11349.3281
Epoch 575/10000
1/1 [==============================] - 0s 4ms/step - loss: 27735.6953
Epoch 576/10000
1/1 [==============================] - 0s 4ms/step - loss: 16138.0615
Epoch 577/10000
1/1 [==============================] - 0s 4ms/step - loss: 24851.7188
Epoch 578/10000
1/1 [==============================] - 0s 4ms/step - loss: 9566.1396
Epoch 579/10000
1/1 [==============================] - 0s 4ms/step - loss: 12892.5732
Epoch 580/10000
1/1 [==============================] - 0s 4ms/step - loss: 16046.2266
Epoch 581/10000
1/1 [==============================] - 0s 4ms/step - loss: 23172.5156
Epoch 582/10000
1/1 [==============================] - 0s 4ms/step - loss: 10440.3389
Epoch 583/10000
1/1 [==============================] - 0s 6ms/step - lo

1/1 [==============================] - 0s 4ms/step - loss: 23323.0762
Epoch 668/10000
1/1 [==============================] - 0s 4ms/step - loss: 16843.5039
Epoch 669/10000
1/1 [==============================] - 0s 4ms/step - loss: 13532.7236
Epoch 670/10000
1/1 [==============================] - 0s 4ms/step - loss: 18651.2500
Epoch 671/10000
1/1 [==============================] - 0s 4ms/step - loss: 13835.9307
Epoch 672/10000
1/1 [==============================] - 0s 4ms/step - loss: 18147.5391
Epoch 673/10000
1/1 [==============================] - 0s 4ms/step - loss: 20606.1660
Epoch 674/10000
1/1 [==============================] - 0s 4ms/step - loss: 27458.4824
Epoch 675/10000
1/1 [==============================] - 0s 4ms/step - loss: 13184.7393
Epoch 676/10000
1/1 [==============================] - 0s 4ms/step - loss: 11708.2617
Epoch 677/10000
1/1 [==============================] - 0s 4ms/step - loss: 22760.0938
Epoch 678/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 13873.9932
Epoch 763/10000
1/1 [==============================] - 0s 4ms/step - loss: 22784.8340
Epoch 764/10000
1/1 [==============================] - 0s 4ms/step - loss: 6329.0688
Epoch 765/10000
1/1 [==============================] - 0s 4ms/step - loss: 5731.4937
Epoch 766/10000
1/1 [==============================] - 0s 4ms/step - loss: 12318.5195
Epoch 767/10000
1/1 [==============================] - 0s 4ms/step - loss: 22558.4609
Epoch 768/10000
1/1 [==============================] - 0s 3ms/step - loss: 14712.6846
Epoch 769/10000
1/1 [==============================] - 0s 4ms/step - loss: 32080.8281
Epoch 770/10000
1/1 [==============================] - 0s 4ms/step - loss: 12632.4971
Epoch 771/10000
1/1 [==============================] - 0s 4ms/step - loss: 14137.0752
Epoch 772/10000
1/1 [==============================] - 0s 4ms/step - loss: 8796.4453
Epoch 773/10000
1/1 [==============================] - 0s 4ms/step - loss

1/1 [==============================] - 0s 4ms/step - loss: 23196.5723
Epoch 858/10000
1/1 [==============================] - 0s 4ms/step - loss: 13544.7773
Epoch 859/10000
1/1 [==============================] - 0s 4ms/step - loss: 16924.4082
Epoch 860/10000
1/1 [==============================] - 0s 4ms/step - loss: 17120.2324
Epoch 861/10000
1/1 [==============================] - 0s 4ms/step - loss: 19056.7676
Epoch 862/10000
1/1 [==============================] - 0s 4ms/step - loss: 18771.1816
Epoch 863/10000
1/1 [==============================] - 0s 4ms/step - loss: 10657.8955
Epoch 864/10000
1/1 [==============================] - 0s 4ms/step - loss: 12826.8096
Epoch 865/10000
1/1 [==============================] - 0s 4ms/step - loss: 16815.1816
Epoch 866/10000
1/1 [==============================] - 0s 4ms/step - loss: 8989.4482
Epoch 867/10000
1/1 [==============================] - 0s 4ms/step - loss: 10313.3848
Epoch 868/10000
1/1 [==============================] - 0s 5ms/step - lo

1/1 [==============================] - 0s 4ms/step - loss: 9110.2803
Epoch 953/10000
1/1 [==============================] - 0s 4ms/step - loss: 8634.8975
Epoch 954/10000
1/1 [==============================] - 0s 4ms/step - loss: 8855.2344
Epoch 955/10000
1/1 [==============================] - 0s 4ms/step - loss: 10781.2158
Epoch 956/10000
1/1 [==============================] - 0s 4ms/step - loss: 12735.1240
Epoch 957/10000
1/1 [==============================] - 0s 4ms/step - loss: 27607.5098
Epoch 958/10000
1/1 [==============================] - 0s 4ms/step - loss: 5805.2983
Epoch 959/10000
1/1 [==============================] - 0s 4ms/step - loss: 22592.6543
Epoch 960/10000
1/1 [==============================] - 0s 4ms/step - loss: 15504.0186
Epoch 961/10000
1/1 [==============================] - 0s 4ms/step - loss: 12805.0273
Epoch 962/10000
1/1 [==============================] - 0s 5ms/step - loss: 13724.5312
Epoch 963/10000
1/1 [==============================] - 0s 6ms/step - loss:

In [27]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 571ms/step
[2022.411](test_target) - [1967.8309](prediction) = 54.58006738281256


In [28]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.026987623872107378

In [29]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [30]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Produção de Cimento (t)
0,-0.969522,-1.359398,-1.050601,-0.981735,-1.252557,-1.264163,-1.233150,1.148205,-1.122043,-1.121418,...,-0.449079,0.959570,1.349075,1.396668,1.302745,0.665154,-1.232563,-1.223094,1.316781,-1.091916
1,-0.406879,0.342020,-0.294564,-0.390692,0.057661,0.083064,0.016987,0.140893,-0.184474,-0.185492,...,1.385895,0.419892,-0.307116,-0.890660,-0.174780,0.748246,0.015789,-0.003295,-0.211662,-0.232366
2,1.376400,1.017379,1.345165,1.372427,1.194896,1.181098,1.216163,-1.289098,1.306518,1.306910,...,-0.936816,-1.379462,-1.041959,-0.506008,-1.127966,-1.413400,1.216774,1.226389,-1.105119,1.324283


train_target:


,Pernambuco - Consumo de Cimento (t)
0,1878.938
1,2406.931
2,2496.113


test_input:


...

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Produção de Cimento (t)
12,0.243152,-0.752383,-1.854042,-1.166663,0.818285,0.699365,1.038092,1.108405,2.635823,-0.189959,...,0.742044,-1.580713,-1.117166,-0.348766,-0.031355,0.055743,1.110593,1.231105,0.854739,-1.391209


test_target:


,Pernambuco - Consumo de Cimento (t)
12,2022.411


1/1 [==============================] - 1s 550ms/step
Error: 54.58006738281256




[2467.93310546875,
 2601.56298828125,
 2143.88818359375,
 2711.078369140625,
 2672.072998046875,
 2300.201171875,
 1857.5345458984375,
 1706.0999755859375,
 1661.4375,
 1967.8309326171875]

In [31]:
display(targets)
display(predictions)

[2728.23,
 2982.021,
 2983.632857142857,
 2632.8378857142857,
 2308.455257142857,
 1813.941,
 1645.004,
 1690.846,
 2018.834,
 2022.411]

[2467.93310546875,
 2601.56298828125,
 2143.88818359375,
 2711.078369140625,
 2672.072998046875,
 2300.201171875,
 1857.5345458984375,
 1706.0999755859375,
 1661.4375,
 1967.8309326171875]

In [32]:
mae = mean_absolute_error(predictions, targets)
mae

304.83790648716524

In [33]:
porcentage = mae/np.mean(targets)
porcentage

0.133547297787489